In [ ]:
import pickle
import random

import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import scipy as sc
from scipy.spatial import Delaunay
from scipy.spatial.distance import cdist

from simplicial_kuramoto import SimplicialComplex
from simplicial_kuramoto.graph_generator import modular_graph
from simplicial_kuramoto.integrators import *
from simplicial_kuramoto.plotting import *

In [ ]:
def get_delauney_holes_multi(n_points, centre_holes, radius, points=[]):

    if len(points) == 0:
        x = np.random.rand(n_points)
        y = np.random.rand(n_points)
        points = np.vstack([x, y]).T

    tri = Delaunay(points)

    edge_list = []

    idx_inside = np.empty([0], dtype=int)
    for i in range(centre_holes.shape[0]):
        idx_inside = np.hstack(
            [idx_inside, encloses([centre_holes[i]], points, radius)[1]]
        )

    for t in tri.simplices:

        if t[0] not in idx_inside and t[1] not in idx_inside:
            edge_list.append([t[0], t[1]])

        if t[1] not in idx_inside and t[2] not in idx_inside:
            edge_list.append([t[1], t[2]])

        if t[0] not in idx_inside and t[2] not in idx_inside:
            edge_list.append([t[0], t[2]])

    graph = nx.Graph()
    # graph.add_nodes_from(np.arange(len(points)))
    graph.add_edges_from(edge_list)

    Gcc = sorted(nx.connected_components(graph), key=len, reverse=True)
    g = graph.subgraph(Gcc[0])

    return g, points


def encloses(centre, points, radius):
    inside_hole = cdist(centre, points, "euclidean") <= radius
    idx_inside = np.where(inside_hole)

    return idx_inside

In [ ]:
centre_holes = np.array([[0.25, 0.5], [0.75, 0.5]])
n_points = 20
radius = 0.2
graph, points = get_delauney_holes_multi(n_points, centre_holes, radius)
complex_delaunay = SimplicialComplex(graph=graph, no_faces=False)

phase = np.random.uniform(0, 2 * np.pi, len(graph.edges))
phase2 = np.tile(phase, 2)

B0 = complex_delaunay.node_incidence_matrix
B1 = complex_delaunay.edge_incidence_matrix

Nn = B0.shape[1]
Ne = B0.shape[0]
Nf = B1.shape[0]

Ie = np.diag(np.ones(Ne))
nIe = np.diag(-np.ones(Ne))

V = np.concatenate((Ie, nIe), axis=0)
hB0 = V.dot(B0.toarray())
hB1 = B1.toarray().dot(V.T)

hB0p = hB0.copy()
hB0n = hB0.copy()

hB0p[hB0p < 0] = 0
hB0n[hB0n > 0] = 0
hB0n = np.negative(hB0n)

hB1p = hB1.copy()
hB1n = hB1.copy()
hB1p[hB1p < 0] = 0
hB1n[hB1n > 0] = 0
hB1n = np.negative(hB1n)

print(np.sin(B0.T.dot(phase)).shape)
print(np.sin(hB0p.T.dot(phase2)).shape)
print(np.sin(B0.T.dot(phase)))
print(np.sin(hB0p.T.dot(phase2)))